# Stock Data Exploration

This notebook demonstrates how to explore stock data before forecasting.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
from src.data import Fetcher, prepare_for_prophet

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Fetch Stock Data

In [ ]:
# Initialize fetcher
fetcher = Fetcher()

# Fetch data
symbol = 'AAPL'
start = '2023-01-01'
end = '2024-10-31'

print(f"Fetching {symbol} data...")
data = fetcher.fetch(symbol, start, end)
print(f"✅ Fetched {len(data)} records")

## 2. Explore Raw Data

In [ ]:
# Show first rows
print("First 5 rows:")
data.head()

In [ ]:
# Basic statistics
print("Statistics:")
data.describe()

In [ ]:
# Plot closing price
plt.figure(figsize=(14, 6))
plt.plot(data.index, data['close'], linewidth=2)
plt.title(f'{symbol} Closing Price', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Prepare for Prophet

In [ ]:
# Convert to Prophet format
prophet_data = prepare_for_prophet(data)
print(f"Prophet data shape: {prophet_data.shape}")
prophet_data.head()

In [ ]:
# Plot Prophet data
plt.figure(figsize=(14, 6))
plt.plot(prophet_data['ds'], prophet_data['y'], linewidth=2)
plt.title(f'{symbol} - Ready for Prophet', fontsize=16)
plt.xlabel('Date (ds)', fontsize=12)
plt.ylabel('Price (y)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Analyze Trends

In [ ]:
# Calculate rolling statistics
prophet_data['rolling_mean_30'] = prophet_data['y'].rolling(window=30).mean()
prophet_data['rolling_std_30'] = prophet_data['y'].rolling(window=30).std()

# Plot with rolling statistics
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Price with rolling mean
ax1.plot(prophet_data['ds'], prophet_data['y'], label='Price', alpha=0.7)
ax1.plot(prophet_data['ds'], prophet_data['rolling_mean_30'], 
         label='30-day MA', linewidth=2, color='red')
ax1.set_title(f'{symbol} Price with 30-day Moving Average', fontsize=14)
ax1.set_ylabel('Price (USD)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rolling standard deviation (volatility)
ax2.plot(prophet_data['ds'], prophet_data['rolling_std_30'], 
         linewidth=2, color='orange')
ax2.set_title('30-day Rolling Volatility', fontsize=14)
ax2.set_xlabel('Date')
ax2.set_ylabel('Std Dev')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Compare Multiple Stocks

In [ ]:
# Fetch multiple stocks
symbols = ['AAPL', 'GOOGL', 'MSFT']
stocks_data = {}

for sym in symbols:
    print(f"Fetching {sym}...")
    stocks_data[sym] = fetcher.fetch(sym, start, end)
    
print("✅ All stocks fetched")

In [ ]:
# Normalize prices to compare (start at 100)
plt.figure(figsize=(14, 6))

for sym, data in stocks_data.items():
    normalized = (data['close'] / data['close'].iloc[0]) * 100
    plt.plot(data.index, normalized, label=sym, linewidth=2)

plt.title('Stock Performance Comparison (Normalized)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Normalized Price (Base = 100)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Next Steps

Now that you've explored the data, you can:
1. Run forecasts using `main.py`
2. Try the forecast analysis notebook
3. Experiment with different time periods